In [ ]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
sys.path.append('..')

# Import our modules
from src.basic_operations import *
from src.filters import *
from src.edge_detection import *
from src.segmentation import *
from src.morphology import *
from src.frequency_domain import *
from src.feature_detection import *
from src.deep_learning import *
from src.utils import *

# Configure matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Helper function to display images
def show_comparison(original, processed, titles=['Original', 'Processed']):
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    for ax, img, title in zip(axes, [original, processed], titles):
        if len(img.shape) == 3:
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            ax.imshow(img, cmap='gray')
        ax.set_title(title, fontsize=14)
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

def show_multiple(images, titles, cols=3):
    n = len(images)
    rows = (n + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))
    axes = np.array(axes).flatten()
    
    for i, (ax, img, title) in enumerate(zip(axes, images, titles)):
        if len(img.shape) == 3:
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            ax.imshow(img, cmap='gray')
        ax.set_title(title)
        ax.axis('off')
    
    for ax in axes[n:]:
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Load a sample image
# You can replace this with your own image path
try:
    img = load_image('../test_image/lena.pgm', grayscale=False)
except:
    # Create a sample image if no test image available
    img = np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8)
    # Add some patterns
    cv2.circle(img, (128, 128), 50, (255, 0, 0), -1)
    cv2.rectangle(img, (50, 50), (100, 100), (0, 255, 0), -1)

gray = convert_to_grayscale(img)
print(f"Image shape: {img.shape}")
print(f"Image info: {get_image_info(img)}")

plt.figure(figsize=(8, 6))
if len(img.shape) == 3:
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
else:
    plt.imshow(img, cmap='gray')
plt.title('Sample Image')
plt.axis('off')
plt.show()

## 1. Basic Operations

In [ ]:
# Resize operations
resized_half = resize_image(img, scale=0.5)
resized_double = resize_image(img, scale=2.0)
print(f"Original: {img.shape}, Half: {resized_half.shape}, Double: {resized_double.shape}")

In [ ]:
# Rotation and flipping
rotated = rotate_image(img, 45)
flipped_h = flip_image(img, 'horizontal')
flipped_v = flip_image(img, 'vertical')

show_multiple([img, rotated, flipped_h, flipped_v],
              ['Original', 'Rotated 45°', 'Horizontal Flip', 'Vertical Flip'], cols=4)

In [ ]:
# Brightness and contrast
bright = adjust_brightness(img, 50)
dark = adjust_brightness(img, -50)
high_contrast = adjust_contrast(img, 1.5)
low_contrast = adjust_contrast(img, 0.5)

show_multiple([img, bright, dark, high_contrast, low_contrast],
              ['Original', 'Brighter', 'Darker', 'High Contrast', 'Low Contrast'])

In [ ]:
# Gamma correction
gamma_low = gamma_correction(img, 0.5)  # Brightens
gamma_high = gamma_correction(img, 2.0)  # Darkens

show_multiple([img, gamma_low, gamma_high],
              ['Original', 'Gamma=0.5 (Lighter)', 'Gamma=2.0 (Darker)'])

In [ ]:
# Histogram equalization
equalized = histogram_equalization(img)
clahe = histogram_equalization(img, use_clahe=True)

show_multiple([img, equalized, clahe],
              ['Original', 'Histogram Equalized', 'CLAHE'])

## 2. Filters

In [ ]:
# Smoothing filters
avg = average_filter(img, kernel_size=5)
gauss = gaussian_filter(img, kernel_size=5)
med = median_filter(img, kernel_size=5)
bilat = bilateral_filter(img)

show_multiple([img, avg, gauss, med, bilat],
              ['Original', 'Average', 'Gaussian', 'Median', 'Bilateral'])

In [ ]:
# Sharpening and emboss
sharp = sharpen_filter(img)
unsharp = unsharp_mask(img, amount=1.5)
emboss = emboss_filter(img)

show_multiple([img, sharp, unsharp, emboss],
              ['Original', 'Sharpened', 'Unsharp Mask', 'Emboss'], cols=4)

In [ ]:
# Noise addition and removal
noisy_gauss = add_gaussian_noise(img, std=30)
noisy_sp = add_salt_pepper_noise(img, salt_prob=0.02, pepper_prob=0.02)
denoised = non_local_means_denoise(noisy_gauss)

show_multiple([img, noisy_gauss, noisy_sp, denoised],
              ['Original', 'Gaussian Noise', 'Salt & Pepper', 'Denoised'], cols=4)

## 3. Edge Detection

In [ ]:
# Various edge detection methods
sobel = sobel_edge_detection(gray)
canny = canny_edge_detection(gray, 50, 150)
laplacian = laplacian_edge_detection(gray)
prewitt = prewitt_edge_detection(gray)
roberts = roberts_edge_detection(gray)
scharr = scharr_edge_detection(gray)

show_multiple([gray, sobel, canny, laplacian, prewitt, roberts],
              ['Original', 'Sobel', 'Canny', 'Laplacian', 'Prewitt', 'Roberts'])

In [ ]:
# Auto Canny with different sigma values
auto1 = auto_canny(gray, sigma=0.2)
auto2 = auto_canny(gray, sigma=0.33)
auto3 = auto_canny(gray, sigma=0.5)

show_multiple([gray, auto1, auto2, auto3],
              ['Original', 'Auto Canny σ=0.2', 'Auto Canny σ=0.33', 'Auto Canny σ=0.5'], cols=4)

## 4. Segmentation

In [ ]:
# Thresholding methods
simple = simple_threshold(gray, 127)
otsu, thresh_val = otsu_threshold(gray)
adaptive = adaptive_threshold(gray)

print(f"Otsu optimal threshold: {thresh_val}")
show_multiple([gray, simple, otsu, adaptive],
              ['Original', 'Simple (127)', f'Otsu ({thresh_val})', 'Adaptive'], cols=4)

In [ ]:
# K-means segmentation
kmeans_2 = kmeans_segmentation(img, k=2)
kmeans_4 = kmeans_segmentation(img, k=4)
kmeans_8 = kmeans_segmentation(img, k=8)

show_multiple([img, kmeans_2, kmeans_4, kmeans_8],
              ['Original', 'K=2', 'K=4', 'K=8'], cols=4)

In [ ]:
# Contour detection
contours = contour_detection(gray)
contour_img = draw_contours(img, contours)
print(f"Found {len(contours)} contours")

show_comparison(img, contour_img, ['Original', 'Contours Detected'])

## 5. Morphological Operations

In [ ]:
# Create binary image for morphological operations
_, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

eroded = erosion(binary, kernel_size=5)
dilated = dilation(binary, kernel_size=5)
opened = opening(binary, kernel_size=5)
closed = closing(binary, kernel_size=5)

show_multiple([binary, eroded, dilated, opened, closed],
              ['Binary', 'Erosion', 'Dilation', 'Opening', 'Closing'])

In [ ]:
# Advanced morphological operations
gradient = morphological_gradient(binary)
tophat = top_hat(gray)
blackhat = black_hat(gray)
skeleton = skeletonization(binary)
boundary = boundary_extraction(binary)

show_multiple([binary, gradient, tophat, blackhat, skeleton, boundary],
              ['Binary', 'Gradient', 'Top Hat', 'Black Hat', 'Skeleton', 'Boundary'])

## 6. Frequency Domain

In [ ]:
# DFT and magnitude spectrum
dft = compute_dft(gray)
magnitude = get_magnitude_spectrum(dft)

show_comparison(gray, magnitude, ['Original', 'Magnitude Spectrum'])

In [ ]:
# Low-pass filters comparison
ideal_lp = ideal_lowpass_filter(gray, cutoff=30)
butter_lp = butterworth_lowpass_filter(gray, cutoff=30)
gauss_lp = gaussian_lowpass_filter(gray, cutoff=30)

show_multiple([gray, ideal_lp, butter_lp, gauss_lp],
              ['Original', 'Ideal LP (D0=30)', 'Butterworth LP', 'Gaussian LP'], cols=4)

In [ ]:
# High-pass filter
ideal_hp = ideal_highpass_filter(gray, cutoff=30)

show_comparison(gray, ideal_hp, ['Original', 'Ideal High-Pass (D0=30)'])

In [ ]:
# Homomorphic filter for illumination correction
homomorphic = homomorphic_filter(gray, gamma_l=0.3, gamma_h=2.0, cutoff=30)

show_comparison(gray, homomorphic, ['Original', 'Homomorphic Filter'])

## 7. Feature Detection

In [ ]:
# Corner detection
harris = detect_harris_corners(img)
shi_tomasi, corners = detect_shi_tomasi_corners(img, max_corners=100)

show_multiple([img, harris, shi_tomasi],
              ['Original', 'Harris Corners', 'Shi-Tomasi Corners'])

In [ ]:
# ORB features
orb_img, keypoints, descriptors = detect_orb_features(img, n_features=500)
print(f"Detected {len(keypoints)} ORB keypoints")

show_comparison(img, orb_img, ['Original', 'ORB Features'])

In [ ]:
# Line and circle detection
lines_img, lines = detect_lines_houghp(img)
circles_img, circles = detect_circles_hough(img)

show_multiple([img, lines_img, circles_img],
              ['Original', 'Hough Lines', 'Hough Circles'])

## 8. Deep Learning / Artistic Effects

In [ ]:
# Face detection (if faces present in image)
face_img, faces = detect_faces_haar(img)
print(f"Detected {len(faces)} faces")

show_comparison(img, face_img, ['Original', 'Face Detection'])

In [ ]:
# Artistic effects
pencil = neural_style_transfer_simple(img, style='pencil')
cartoon = neural_style_transfer_simple(img, style='cartoon')
watercolor = neural_style_transfer_simple(img, style='watercolor')
oil = neural_style_transfer_simple(img, style='oil')

show_multiple([img, pencil, cartoon, watercolor, oil],
              ['Original', 'Pencil Sketch', 'Cartoon', 'Watercolor', 'Oil Painting'])

In [ ]:
# Stylization and detail enhancement
styled = stylization(img)
detailed = detail_enhance(img)
gray_sketch, color_sketch = pencil_sketch(img)

show_multiple([img, styled, detailed, color_sketch],
              ['Original', 'Stylized', 'Detail Enhanced', 'Color Pencil Sketch'], cols=4)

## Summary

This notebook demonstrated:

1. **Basic Operations**: Resize, rotate, flip, brightness, contrast, gamma, histogram equalization
2. **Filters**: Average, Gaussian, median, bilateral, sharpen, emboss, denoise
3. **Edge Detection**: Sobel, Canny, Laplacian, Prewitt, Roberts, Scharr
4. **Segmentation**: Thresholding (simple, Otsu, adaptive), K-means, contours
5. **Morphology**: Erosion, dilation, opening, closing, gradient, skeleton
6. **Frequency Domain**: DFT, low/high-pass filters, homomorphic filter
7. **Feature Detection**: Harris, Shi-Tomasi, ORB, Hough transforms
8. **Deep Learning**: Face detection, artistic effects

Use the Streamlit app (`streamlit run app.py`) for an interactive experience!